# jupyter meta-kernel in nodejs?

Yes, please.

TODO: submit to this page an exported notebook of the kernel?

https://github.com/jupyter/jupyter/wiki/Jupyter-kernels

There are 2 main ways to implement a jupyter kernel, by overriding the do_execute, do_inspect, do_complete, etc methods derived from IPython.kernel

https://jupyter-client.readthedocs.io/en/stable/wrapperkernels.html

OR:

https://jupyter-client.readthedocs.io/en/stable/messaging.html#wire-protocol

The way [nel](https://github.com/n-riesco/nel), [jupyter-nodejs](https://github.com/notablemind/jupyter-nodejs), [jp-babel](https://github.com/n-riesco/jp-babel), [jp-kernel](https://github.com/n-riesco/jp-kernel), [ijavascript](https://github.com/n-riesco/ijavascript), and many others in many languages work, by implementing the ZMQ wire protocl, and pretty much calling the same execution flow.


I will attempt to explain the control flow as best I can:

Done: implement both meta kernels in python and socket based kernels 

Done: rewrite entire nodejs kernel functionality in one notebook and a few lines of code

TODO: user input and stdout streams

TODO: make a processing kernel https://github.com/processing/p5.js/wiki/p5.js,-node.js,-socket.io

TODO: use magics parser from jupyter-nodejs, does something with splitting up lines, convert magics to babel and run along side the actual code?

TODO: transpile the kernel in to the native language using AST translations, use command line REPL or native ZMQ

https://jupyter-client.readthedocs.io/en/stable/kernels.html

TODO: use syntax highlighter to "train" a transpiler to write socket based kernels in any language, at least 
BROWSER C# GO JAVA NODE.JS PHP PYTHON RUBY from Google

TODO: connect to M$ lang server


## get kernel json?

The first step of the kernel is to install the definition so it can be recognized by Jupyter. This is a command to run the kernel in it's NATIVE language.

Note: seperation of concerns is always a concern, so I've labeled functions with each specific "template" to achieve a veriety of supported languages.


In [ ]:
var path = require('path');

function jsonInterface(kernel_json) {
    return {
        argv,
        display_name,
        language,
        metadata,
        env,
        interrupt_mode
    } = kernel_json
}

function metaJson(kernel_json) {
    return jsonInterface(Object.assign({
        argv: argv.filter(a => a.includes('{connection_file}')).length === 0
            ? (argv || []).concat(['{connection_file}'])
            : (argv || [])
    }, kernel_json));
}

// TODO: use yargs to parse from string
function pathJson(kernel_json) {
    return jsonInterface(Object.assign({
        argv: (kernel_json.path
               ? [kernel_json.path]
               : []).concat(kernel_json.argv || kernel_json.args || [])
    }, kernel_json))
}

function pythonJson(kernel_json) {
    return pathJson(Object.assign({
        argv: ['python3', '-m', 'IPython.kernel', '-f', '{connection_file}']
    }, kernel_json))
}

// derrived from https://github.com/n-riesco/jp-babel/blob/master/lib/kernel.js

function nodeJson(kernel_json) {
    return pathJson(Object.assign({
        path: process.argv[2] || process.argv[0],
        argv: [kernel_json.path].concat(kernel_json.argv || kernel_json.args || [])
    }, kernel_json))
}

function notebookJson(kernel_json) {
    return jsonInterface(Object.assign({
        argv: ['npm', 'run', 'import', '--', 'get javascript kernel',
               '["{connection_file}"]'],
    }, kernel_json))
}

function bashJson(kernel_json) {
    return jsonInterface(Object.assign({
        argv: ['npm', 'run', 'import', '--', 'get bash kernel',
               '["{connection_file}"]'],
    }, kernel_json))
}

function processingJson(kernel_json) {
    return jsonInterface(Object.assign({
        argv: ['npm', 'run', 'import', '--', 'get processing kernel',
               '["{connection_file}"]'],
    }, kernel_json))
}


module.exports = {
    jsonInterface,
    pathJson,
    pythonJson,
    nodeJson,
    notebookJson,
    bashJson,
    processingJson
};


## jupyter meta kernel?

This meta-kernel runs inside a node process that is started and managed by the wire protocol kernel.

Node specific functionality has been extracted so that it is easy to translate between languages.

Wire meta kernel splits up the meta kernel control flow from the wire request kernel control flow.


In [ ]:
var importer = require('../Core');
var {kernelInfoInterface} = importer.import('get kernel info');
var {notebookJsonTemplate} = importer.import('get kernel json');
var mkdirpSync = importer.import('mkdirp');

function metaKernelInterface(meta_kernel) {
    return Object.assign({
        kernel_info,
        kernel_config,
        do_init, // custom for executing once
        do_message, // custom input message
        do_respond, // custom output message
        do_execute,
        do_complete,
        do_inspect,
        do_history,
        do_is_complete,
        do_shutdown,
        do_install, // custom for executing install script, even remotely!
        // TODO: expansion on interface creates extension in transpiler?
    } = meta_kernel, kernelInfoInterface(meta_kernel.kernel_info));
}

function installKernelJson(configJson) {
    console.log(`installing kernel ${JSON.stringify(configJson)}`);
    mkdirpSync(`./.kernel/${configJson.language}`);
    require('fs').writeFileSync(`./.kernel/${configJson.language}/kernel.json`,
                                JSON.stringify(configJson, null, 4));
// TODO: convert to bash notebook or kernel call
    require('child_process').execSync(`
jupyter kernelspec install --user --replace "./.kernel/${configJson.language}"`);
    require('rimraf').sync('./.kernel');
}

function nativeMetaKernel(meta_kernel) {
    if(typeof meta_kernel.kernel_config === 'function')
        meta_kernel.kernel_config = meta_kernel
            .kernel_config(meta_kernel.kernel_config);
    if(typeof meta_kernel.kernel_info === 'function')
        meta_kernel.kernel_info = meta_kernel
            .kernel_info({
                language_info: meta_kernel.kernel_info.language_info
                    || meta_kernel.language_info
            });
    const kernel = metaKernelInterface(Object.assign({
        language_info: meta_kernel.language_info
            || meta_kernel.kernel_info.language_info,
        do_install: meta_kernel.do_install || installKernelJson,
        do_init: (kernel, config) => {
            console.log('starting meta kernel');
             // incase different than kernel_info.kernel_config
            kernel.kernel_config = config;
            
            if(typeof meta_kernel.do_init === 'function') {
                meta_kernel.do_init(kernel, config);
            }
        },        
        do_message: (kernel, message) => {
            console.log(JSON.stringify(message));
            if(typeof kernel[Object.keys(message)[0]] === 'undefined') {
                console.error(`unhandled message type ${JSON.stringify(message)}`);
                return;
            }
            kernel[Object.keys(message)[0]]
                .apply(kernel, [kernel].concat(Object.values(message)));
        },
        do_is_complete: (kernel, message) => {
            try {
                kernel.do_execute(kernel, message);
                return true;
            } catch (e) {
                return false;
            }
        }
    }, meta_kernel));
    
    if(fs.existsSync(kernel.kernel_config)) {
        kernel.kernel_config = Object.assign(
            {autoinit: true},
            meta_kernel.start_config || {},
            JSON.parse(fs.readFileSync(kernel.kernel_config)),
        );
    }
    if((kernel.kernel_config || {}).autoinit) {
        kernel.do_init(kernel, kernel.kernel_config);
    } 
    // TODO: allow calling any kernel from command line with a repl interface
    // reference jupyter command for this
    else if(meta_kernel.kernel_config === 'do_install') {
        if(typeof kernel.install_config === 'function')
            kernel.install_config = kernel.install_config({
                display_name: kernel.install_config.display_name
                    || kernel.banner,
                language: kernel.install_config.language
                    || kernel.implementation,
            });
        kernel.do_install(kernel.install_config);
    }
    return kernel;
}

module.exports = {
    metaKernelInterface,
    nativeMetaKernel,
};


## process meta kernel?

Add stdio processing to the kernel.


In [ ]:
var importer = require('../Core');
var {nativeMetaKernel} = importer.import('jupyter meta kernel');

function socketMetaKernel(meta_kernel) {
    return nativeMetaKernel(Object.assign({
        do_init: (kernel, config) => {
            if (!kernel.meta_kernel) {
                throw new Error(`meta_kernel not implemented! ${
                                JSON.stringify(meta_kernel)}`)
            }
            
            if(typeof meta_kernel.do_init === 'function') {
                meta_kernel.do_init(kernel, config);
            }
            
            if(kernel.socket) {
                kernel.socket.on('message', do_message);
                kernel.do_respond(kernel, {status: "online"});
            }
        },
        do_respond: (kernel, message) => {
            kernel.socket.send(message);
        },
    }, meta_kernel));
    // TODO: add stdio reporting up through wire kernel
}

function initialize(kernel, config) {
    console.log('spawning child process');
    kernel.socket = require("child_process")
        .spawn(kernel.meta_kernel[0], kernel.meta_kernel.slice(1), {
            cwd: config.cwd || '.',
            stdio: [process.stdin, process.stdout, process.stderr, 'ipc']
        })
}

function processMetaKernel(meta_kernel) {
    var meta_session = Object.assign({
        meta_kernel: (meta_kernel => {
            if(typeof meta_kernel === process) {
                return meta_kernel;
            } else if(typeof meta_kernel === 'object'
               && typeof meta_kernel[0] !== 'undefined') {
                return meta_kernel;
            }
        })(meta_kernel.meta_kernel),
        
        do_init: (kernel, config) => {
            var promise = Promise.resolve();
            if(typeof meta_kernel.do_init === 'function') {
                promise = promise.then(() => meta_kernel.do_init(kernel, config));
            }
            
            if(kernel.meta_kernel === process) {
                kernel.socket = kernel.meta_kernel;
                return;
            }
            
            return promise.then(() => initialize(kernel, config));
        },
        
        do_shutdown: (kernel, message) => {
            return Promise.resolve()
                .then(() => kernel.socket.kill('SIGTERM'))
                .then(() => process.exit() /*request.content.restart
                      ? kernel.do_init(kernel.kernel_config, kernel)
                      : void 0*/)
        },
    }, meta_kernel);
    return socketMetaKernel(meta_session);
}

module.exports = {
    socketMetaKernel,
    processMetaKernel
}


## get kernel language?

language kernel information https://jupyter-client.readthedocs.io/en/stable/messaging.html#kernel-info


In [ ]:

function languageInterface(language_info) {
    return {
        mimetype,
        name,
        file_extension,
        version,
        pygments_lexer,
        codemirror_mode,
        nbconvert_exporter
    } = language_info
}

module.exports = {
    languageInterface
};


## get kernel info?
 
Kernel info is just the static properties required by the meta kernel implementation. These are also returned by the kernel_info_request in the wire protocol implementation



In [3]:

var PACKAGE_VERSION = require('../package.json').version;

function kernelInfoInterface(kernel_info) {
    return {
        protocol_version,
        implementation,
        implementation_version,
        banner,
        language_info,
        help_links,
        install_config // custom, path to it's own script for installing
    } = kernel_info
}

function nativeKernelInfo(kernel_info) {
    if(typeof kernel_info.language_info === 'function')
        kernel_info.language_info = kernel_info.language_info(kernel_info.language_info);
    return kernelInfoInterface(Object.assign({
        protocol_version: kernel_info.protocol_version || '5.1',
        implementation: kernel_info.implementation || kernel_info.language_info.name,
        implementation_version: kernel_info.implementation_version || PACKAGE_VERSION,
        banner: kernel_info.banner || kernel_info.language_info.language,
        language_info: Object.assign(kernel_info.language_info || {}, {
            name: kernel_info.language_info.name || kernel_info.implementation
        })
    }, kernel_info));
}

module.exports = {
    kernelInfoInterface,
    nativeKernelInfo
};


{ kernelInfoInterface: [Function: kernelInfoInterface],
  nativeKernelInfo: [Function: nativeKernelInfo] }

## jupyter wire kernel?


partly derrived from:

https://github.com/n-riesco/nel/blob/master/lib/nel.js

https://github.com/n-riesco/ijavascript/blob/master/lib/kernel.js

https://github.com/notablemind/jupyter-nodejs/blob/master/lib/kernel.js

https://github.com/n-riesco/jp-kernel/blob/master/lib/jp-kernel.js

these functions are split up to help distinguish between control flow and language specific functionality.

if the wire kernel methods are not defined, the response is sent directly from the native kernel to the front-end



In [ ]:
var importer = require('../Core');
var {
    setupSockets, parseMessage, collapseMessage
} = importer.import('decode encode ipython zmq protocol');
var {metaKernelInterface} = importer.import('jupyter meta kernel');
var {processMetaKernel} = importer.import('process meta kernel');
var {
    do_is_complete, do_execute, do_complete,
    do_history, do_inspect, do_shutdown
} = importer.import('wire meta kernel');

function wireKernelInterface(session) {
    return Object.assign({
        // implement all requests
        execute_request, inspect_request, complete_request,
        history_request, kernel_info_request, is_complete_request,
        connect_request, comm_info_request, kernel_info_request,
        shutdown_request, interrupt_request, input_request,
        
        // implement all replys
        execute_reply, inspect_reply, complete_reply,
        history_reply, kernel_info_reply, is_complete_reply,
        connect_reply, comm_info_reply, kernel_info_reply,
        shutdown_reply, interrupt_reply, input_reply,
        
        // a few extra protocol methods
        display_data, update_display_data, execute_input,
        execute_result, error, status, clear_output,
        comm_msg, comm_close,
        
    } = session, metaKernelInterface(session));
}

// TODO: move this to patterns
function addCB(og, cb) {
    return cb(og.apply(null, Array.from(arguments).slice(2)));
}

function bindSocket(socket, kernel, config) {
    return socket
        .on('message', addCB.bind(null, parseMessage, parsed => {
            // TODO: need to pass arguments, that's why callback was used
            parsed.respond = (message) => kernel.do_respond(
                kernel, message,
                collapseMessage(config.key, parsed.header, parsed.metadata || {},
                                message))
            return kernel.do_message(kernel, parsed)
        }));
}

function wireRespond(kernel, message, encoded) {
    console.log(`response ${JSON.stringify(message)}`);
    if(Object.keys(message)[0] === 'shutdown_reply')
        kernel.sockets.control.send(encoded);
    else if(Object.keys(message)[0].substr(-6) === '_reply')
        kernel.sockets.shell.send(encoded);
    else if(Object.keys(message)[0] === 'input_request')
        kernel.sockets.stdin.send(encoded);
    else
        kernel.sockets.iopub.send(encoded);
}

function wireMessage(kernel, message) {
// TODO: add execution count recorder, also to do_respond maybe
    /*
    
    // assign the responder
    var result = {};
    var execution_count = Object.values(message)[0].execution_count;
    delete Object.values(message)[0].execution_count;
    result[Object.keys(message)[0]] = {
        execution_count,
        content: Object.values(message)[0],
        respond: responders[execution_count]
    };

    */
}

function wireKernel(session) {
    var meta_session = wireKernelInterface(Object.assign({
        // TODO: enjoy this pattern, can't be replace by child class
        do_init: (kernel, config) => {
            console.log('starting wire kernel');
            return setupSockets(config).then(sockets => {
                kernel.sockets = sockets;
                console.log('connecting sockets');
                sockets.heartbeat.on('message', sockets.heartbeat.send);
                [sockets.control, sockets.shell, sockets.stdin]
                    .forEach(socket => bindSocket(socket, kernel, config))
                // iopub appears to be write to only

                // calls child class after
                if(typeof session.do_init === 'function') {
                    session.do_init(kernel, config);
                }
            });
        },
        do_respond: wireRespond,
        do_message: wireMessage,
        // TODO: bubble response messages from child to front-end
        status: (kernel, request) => kernel.do_respond(kernel, {
            status: request.content
        }),
        input_request: (kernel, request) => {
            // TODO: finish this
            //this.onReplies[response.header.msg_id] = onReply;
        },
        comm_info_request: (kernel, request) => {
            request.respond({status: {execution_state: 'busy'}});
            request.respond({status: {execution_state: 'idle'}});
            request.respond({comm_info_reply: {comms: {}}});
        },
        kernel_info_request: (kernel, request) => {
            request.respond({status: {execution_state: 'busy'}});
            request.respond({status: {execution_state: 'idle'}});
            request.respond({kernel_info_reply: kernel.kernel_info});
        },
        
        shutdown_request: do_shutdown,
        is_complete_request: do_is_complete,
        execute_request: do_execute,
        complete_request: do_complete,
        history_request: do_history,
        inspect_request: do_inspect,
        
        do_execute: (kernel, message) =>
            kernel.socket.send({do_execute: message}),
        do_complete: (kernel, message) =>
            kernel.socket.send({do_complete: message}),
        do_inspect: (kernel, message) =>
            kernel.socket.send({do_inspect: message}),
        do_history: (kernel, message) =>
            kernel.socket.send({do_history: message}),
        do_is_complete: (kernel, message) =>
            kernel.socket.send({do_is_complete: message}),
        // TODO: do something on the client when shutting down
        //   same thing as parent but with process?
        // do_shutdown: (kernel, message) => kernel.socket.send({do_shutdown: message}),
        
    }, session));
    return processMetaKernel(meta_session);
}

module.exports = {
    wireKernelInterface,
    wireKernel
}


## wire meta kernel?

converts wire kernel control to meta kernel messaging.


In [ ]:
var count = 0;
var responders = [];

function do_execute(kernel, request) {
    var execution_count = ++count;
    responders[execution_count] = request.respond;
    request.respond({status: {execution_state: 'busy'}});
    request.respond({execute_input: {
        execution_count: request.execution_count,
        code: request.content.code,
    }})
    var result;
    return Promise.resolve()
        .then(() => kernel.do_execute(request.content))
        .then(r => result = r)
        .then(() => request.respond({status: {execution_state: 'idle'}}))
        .then(() => request.respond({execute_reply: {
            status: 'ok',
            execution_count: request.execution_count,
            payload: [], // TODO(NR) not implemented,
            user_expressions: {}, // TODO(NR) not implemented,
        }}))
        .then(() => request.respond({execute_result: {
            execution_count: request.execution_count,
            data: {'text/plain': result + ''},
            metadata: {}
        }}))
}

function do_display(kernel, request) {
    request.respond(request.display_id
        ? {update_display_data: {
            metadata: {},
            data: request.content,
            transient: request.display_id}}
        : {display_data: {metadata: {}, data: request.content}})
}

function do_shutdown(kernel, request) {
    request.respond({status: {execution_state: 'busy'}});
    return Promise.resolve()
        .then(() => kernel.do_shutdown(request.content))
        .then(r => result = r)
        // TODO: some sort of shutdown scripting
        .then(() => request.respond({status: {execution_state: 'idle'}}))
        .then(() => request.respond({shutdown_reply: request.content}))
}

function do_complete(kernel, request) {
    request.respond({status: {execution_state: 'busy'}});
    var result;
    return Promise.resolve()
        .then(() => kernel.do_complete(request.content))
        .then(r => result = r)
        .then(() => request.respond({status: {execution_state: 'idle'}}))
        .then(() => request.respond({complete_reply: {
            matches: result.completion.list,
            cursor_start: result.completion.cursorStart,
            cursor_end: result.completion.cursorEnd,
            status: "ok",
        }}))
}

function do_history(kernel, request) {
    request.respond({status: {execution_state: 'busy'}});
    return Promise.resolve()
        .then(() => kernel.do_history(request.content))
        .then(r => result = r)
        .then(() => request.respond({status: {execution_state: 'idle'}}))
        .then(() => request.respond({history_reply: {
            history: [] // TODO
        }}))
}

function do_is_complete(kernel, request) {
    request.respond({status: {execution_state: 'busy'}});
    var result;
    return Promise.resolve()
        .then(() => kernel.do_is_complete(request.content))
        .then(r => result = r)
        .then(() => request.respond({status: {execution_state: 'idle'}}))
        .then(() => request.respond({is_complete_reply: {
            status: (result ? 'complete': 'incomplete'),
            indent: ''
        }}))
}

function do_inspect(kernel, request) {
    var execution_count = ++count;
    request.respond({status: {execution_state: 'busy'}});
    var result;
    return Promise.resolve()
        .then(() => kernel.do_inspect(request.content))
        .then(r => result = r)
        .then(() => request.respond({status: {execution_state: 'idle'}}))
        // TODO: move this to do_display method?
        .then(() => result.inspection
              ? result.inspection.type + ': ' + result.inspection.string
              : result.doc.usage
                  ? result.doc.usage + '\n\n' + result.doc.description
                  : result)
        // TODO: use error for all promises
        .then(() => request.respond({inspect_reply: {
            found: true,
            data: {'text/plain': result, 'text/html': `<pre>${result}</pre>`},
            metadata: {},
            status: "ok",
        }}))
        .catch(e => request.respond({inspect_reply: {
            status: "error",
            execution_count: execution_count,
            ename: result.error.ename,
            evalue: result.error.evalue,
            traceback: result.error.traceback,
        }}))
}

function wireMetaKernel(meta_kernel) {
    return metaKernelInterface(Object.assign({
        do_message,
        do_shutdown,
        do_is_complete,
        do_complete,
        do_history,
        do_inspect,
    }, meta_kernel))
}

module.exports = {
    wireMetaKernel
}


## get javascript kernel?

derrived from:

https://github.com/n-riesco/jp-babel/blob/master/lib/kernel.js



In [ ]:
var importer = require('../Core');
var {nodeJson, notebookJson} = importer.import('get kernel json');
var {wireKernel} = importer.import('jupyter wire kernel');
var {nativeKernelInfo} = importer.import('get kernel info');
var {processMetaKernel} = importer.import('process meta kernel');
var {languageInterface} = importer.import('get kernel language');

function getVersion(str) {
    return str.split('.').map(v => parseInt(v, 10)).join('.')
}

function nodeKernelInfo(kernel_info) {
    return nativeKernelInfo(Object.assign({
        banner: 'Node JS',
        // TODO: automatically create this from installation intructions
        help_links: ['https://nodejs.org']
    }, kernel_info));
}

function nodeMetaKernel(meta_kernel) {
    return processMetaKernel(Object.assign({
        do_execute: (kernel, request) => {
            var result = new require('vm').runInThisContext(kernel.transpile(code));
            return result;
        },
        do_init: (kernel, config) => {
            kernel.transpile = require("@babel/core").transform.bind({
                presets: [
                    [require.resolve("@babel/preset-env"), {
                        loose: true,
                        targets: {node: true},
                    }],
                ],
            });
        }
        // do_is_complete: compileFunction instead of execute to get any parsing errors
    }, meta_kernel));
}

function nodeLanguage(language_info) {
    return languageInterface(Object.assign({
        name: 'node',
        version: getVersion(process.versions.node),
        file_extension: '.js',
        mimetype: 'application/javascript',
        codemirror_mode: 'javascript'
    }, language_info));
}

function nodeKernel(config, options) {
    var kernel = wireKernel({
        kernel_config: config,
        start_config: options,
        install_config: notebookJson,
        kernel_info: nodeKernelInfo,
        language_info: nodeLanguage,
        meta_kernel: (meta_kernel => {
            if(typeof meta_kernel[0] !== 'undefined') {
                return meta_kernel;
            } else if(typeof meta_kernel === 'object') {
                return [process.argv[0], '--eval', `
((kernel) => kernel.do_init(Object.assign(kernel, {meta_kernel: process})))
({${Object.keys(meta_kernel)
                .map(k => `'${k}': (${meta_kernel[k].toString()})`)
                .join(',')}})`];
            } else if (typeof meta_kernel === 'function') {
                return [process.argv[0], '--eval', meta_kernel.toString()];
            } else if (typeof meta_kernel === 'string') {
                return [process.argv[0], '--eval', meta_kernel];
            }
            throw new Error(`meta_kernel not implemented! ${
                            JSON.stringify(meta_kernel.meta_kernel)}`);
        })(nodeMetaKernel),
    });
    return kernel;
}

module.exports = nodeKernel;
module.exports.nodeMetaKernel = nodeMetaKernel;

if(typeof $$ !== 'undefined') {
    $$.sendResult(nodeKernel());
}



Long-term create a jupyter kernel for any REPL interface

looks like it is just passed to the command line in rust

https://github.com/google/evcxr/blob/602db3ef52bfddeb34608877bd72b9d0d112fa26/evcxr/src/module.rs

https://github.com/JuliaLang/IJulia.jl


Fix ijavascript not showing filename

https://nodejs.org/api/vm.html#vm_class_vm_script

Fix the problem with ijavascript not supporting %%

Go through this list and find demo code for every language and make sure it works with our REPL, minimize dependencies


Add HTML, CSS/SCSS kernels with some sort of visual output specially make for the file type

Create a kernels notebook




In [ ]:

// TODO: derrive from 
//  https://github.com/takluyver/bash_kernel/blob/master/bash_kernel/kernel.py
function bashKernel () {
    
}

function bashLanguage() {
    return languageInterface(Object.assign({
        name: 'bash',
        // TODO: use the language kernel to execute instead
        version: require('child_process').execSync(`
bash --version | grep "bash" | cut -f 4 -d " " | cut -d "-" -f 1  | cut -d "(" -f 1
`).toString().trim(),
        file_extension: '.sh',
        mimetype: 'text/x-sh',
        codemirror_mode: 'shell'
    }, language_info));
}


## decode encode ipython zmq protocol?

translate between standard objects and the wire protocol message, using the first key of the object as the message type like `{execute_request: {content: etc}}`

In [ ]:
var util = require('util');
var zmq = require("jmp").zmq;
var crypto = require('crypto');
var uuid = require('uuid/v4');

// uuids, delim, hmac, header, parent_header, metadata, content
var DELIM = '<IDS|MSG>'
function parseMessage() {
    const strs = [].map.call(arguments, a => a.toString())
    let i
    for (i=0; i<strs.length; i++) {
      if (strs[i] === DELIM) {
        break
      }
    }
    const uuids = [].slice.call(arguments, 0, i)
    const args = strs.slice(i + 2).map(a => JSON.parse(a))
    let [header, parent, metadata, content] = args
    var result = {};
    result[header.msg_type] = {content, header, metadata, parent};
    return result;
}

function hash(string, key) {
    const hmac = crypto.createHmac('sha256', key)
    hmac.update(string)
    const res = hmac.digest('hex')
    return res
}

function json(data) {
    return JSON.stringify(data).replace('\ufdd0', '\\ufdd0')
}

function collapseMessage(key, parent, metadata, content) {
    var header = Object.assign({}, parent, {
        msg_id: uuid(),
        msg_type: Object.keys(content)[0]
    })
    const toHash = [
      json(header),
      json(parent),
      json(metadata || {}),
      ...Object.values(content).map(a => json(a))]
    const hmac = hash(toHash.join(''), key)
    return parent.uuids.concat([DELIM, hmac]).concat(toHash);
}

function setupSockets(config) {
    const sockets = {
        control: {
            port: config.control_port,
            type: 'xrep',
        },
        shell: {
            port: config.shell_port,
            type: 'xrep',
        },
        stdin: {
            port: config.stdin_port,
            type: 'router',
        },
        iopub: {
            port: config.iopub_port,
            type: 'pub',
        },
        heartbeat: {
            port: config.hb_port,
            type: 'rep',
        }
    }
    var keys = Object.keys(sockets);
    return Promise.all(keys.map(s => setupSocket.apply(null, [sockets[s], config])))
        .then(sockets => sockets.reduce((obj, socket, i) =>
                                        (obj[keys[i]] = socket, obj), {}))
}

function setupSocket(config, general) {
    const sock = zmq.socket(config.type);
    const addr = general.transport + '://' + general.ip + ':' + config.port
    return util.promisify(sock.bind.bind(sock))(addr)
        .then(() => sock)
}

module.exports = {
    parseMessage,
    collapseMessage,
    setupSockets
}